### Deep dream in PyTorch

However, you can also use any model from VGG family from torchvision.models.

Please note, that any other model from torchvision.models might not work properly with this code.

That's what happens when you play with some ideas, that are proposed [here](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html).

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.ndimage as nd
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models, transforms
from tqdm.auto import tqdm

In [22]:
def tensor_to_image(tensor):
    img = tensor[0].detach().cpu()
    img = img.permute(1, 2, 0).numpy()
    return img

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [194]:
import os
os.makedirs('./data/', exist_ok=True)
!wget -q https://i.kym-cdn.com/entries/icons/mobile/000/030/157/womanyellingcat.jpg -O data/my_img.jpg
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/450px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg -O data/art.jpg

# Let's Dream

In [81]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
preprocess = transforms.Normalize(mean, std)


def denormalize(image_np):
    image_np = image_np[0]
    image_np = image_np.transpose(1, 2, 0)
    return np.clip(image_np, 0.0, 1.0)


def dream_with_model(image: np.ndarray, model: nn.Module, iterations: int, lr: float=0.01):
    """ Updates the image to maximize outputs for n iterations """
    # set up trainable input tensor
    in_tensor = torch.tensor(image, dtype=torch.float32, device=device, requires_grad=True)
    
    # apply data normalization to match preprocessing from pretrained model
    in_tensor.data = preprocess(in_tensor.data)
    
    for i in range(iterations):
        model.zero_grad()
        out = model(in_tensor)
        
        loss = out.abs().sum() # .norm()
        loss.backward()
        
        avg_grad = in_tensor.grad.abs().cpu().mean().item()
        norm_lr = lr / avg_grad
        
        in_tensor.data += norm_lr * in_tensor.grad
        in_tensor.data.clamp_(-2.0, 2.0)
        in_tensor.grad.zero_()
    output = in_tensor.detach().cpu().numpy()
    output = output * std[None, :, None, None] + mean[None, :, None, None]
    return output


def deep_dream(image, model, iterations, lr, octave_scale, num_octaves):
    """ Main Deep Dream processing """
    np_image_bchw = image.transpose(2, 0, 1)[None] / 255.

    # Extract image representations for each octave
    octaves = [np_image_bchw]
    for idx in range(num_octaves - 1):
        zoom = (1, 1, 1 / octave_scale, 1 / octave_scale)
        zoomed_octave = nd.zoom(octaves[-1], zoom=zoom, order=1)
        octaves.append(zoomed_octave)

    detail = np.zeros_like(octaves[-1])
    for octave, octave_base in enumerate(octaves[::-1]):
        if octave > 0:
            # Upsample detail to new octave dimension
            octave_inv_zoom = np.array(octave_base.shape) / np.array(detail.shape)
            detail = nd.zoom(detail, zoom=octave_inv_zoom, order=1)
        
        # Add deep dream detail from previous octave to new base
        input_image = octave_base + detail
        # Get new deep dream image
        dreamed_image = dream_with_model(input_image, model, iterations, lr)
        # Extract deep dream details
        detail = dreamed_image - octave_base
    
    return denormalize(dreamed_image)

In [82]:
# you can play with the parameters
layer_num = 32
iterations = 100
octave_scale = 1.3
num_octaves = 2
lr = 0.01

In [83]:
# read the data
dst_size = (512, 256) # (w, h)
image = Image.open("data/my_img.jpg").resize(dst_size)
# image = Image.open("data/art.jpg").resize(dst_size)
image = np.array(image)

# Define the model
network = models.vgg19(pretrained=True)
layers = list(network.features.children())

model = nn.Sequential(*layers[: (layer_num + 1)]).to(device)

In [84]:
dreamed_image = deep_dream(
    image,
    model,
    iterations=iterations,
    lr=lr,
    octave_scale=octave_scale,
    num_octaves=num_octaves
)

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(dreamed_image)
plt.show()